## Training

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from silence_tensorflow import silence_tensorflow
silence_tensorflow()

import argparse
from datetime import date
from glob import glob
import os
import sys
import numpy as np
import tensorflow as tf
import pandas as pd

from tensorflow import keras
import tensorflow.keras.backend as K
from tensorflow.keras.optimizers import Adam, SGD

from augmentor.color import VisualEffect
from augmentor.misc import MiscEffect
from model import efficientdet
from losses import smooth_l1, focal, smooth_l1_quad, iou_loss
from efficientnet import BASE_WEIGHTS_PATH, WEIGHTS_HASHES

from train import *

In [3]:
# train all layers
args = "\
--snapshot imagenet \
--snapshot-path checkpoints/gbb_cat_dog \
--loss piou_l3 \
--regression_weight 10 \
--phi 0 \
--weighted-bifpn \
--gpu 0,1 \
--epochs 200 \
--no-evaluation \
--compute-val-loss \
--lr 5e-4 \
--batch-size 16 \
--random-transform \
--use_tfrecords \
csv --annotations_path /datasets/dataset/coco2017/annotations/instances_train2017_obb_cat_dog.csv \
--base_dir_train /datasets/dataset/coco2017/train2017 \
--val_annotations_path /datasets/dataset/coco2017/annotations/instances_val2017_obb_cat_dog.csv \
--base_dir_val /datasets/dataset/coco2017/val2017 \
--classes_path /datasets/dataset/coco2017/annotations/classes_cat_dog.csv"

# parse arguments
args = parse_args(args.split(' '))

{'dataset_type': 'csv', 'detect_quadrangle': False, 'detect_text': False, 'snapshot': 'imagenet', 'freeze_backbone': False, 'freeze_bn': False, 'weighted_bifpn': True, 'lr': 0.0005, 'batch_size': 16, 'phi': 0, 'gpu': '0,1', 'epochs': 200, 'steps': 10000, 'snapshot_path': 'checkpoints/gbb_cat_dog', 'tensorboard_dir': 'logs/2021-04-04', 'snapshots': True, 'evaluation': False, 'random_transform': True, 'compute_val_loss': True, 'loss': 'piou_l3', 'regression_weight': 10.0, 'use_tfrecords': True, 'use_classweights': False, 'freeze_iterations': 0, 'steps_per_epoch': 0, 'multiprocessing': False, 'workers': 1, 'max_queue_size': 10, 'annotations_path': '/datasets/dataset/coco2017/annotations/instances_train2017_obb_cat_dog.csv', 'base_dir_train': '/datasets/dataset/coco2017/train2017', 'classes_path': '/datasets/dataset/coco2017/annotations/classes_cat_dog.csv', 'val_annotations_path': '/datasets/dataset/coco2017/annotations/instances_val2017_obb_cat_dog.csv', 'base_dir_val': '/datasets/datase

In [4]:
# optionally choose specific GPU
if args.gpu:
    os.environ['CUDA_VISIBLE_DEVICES'] = args.gpu
    
if len(tf.config.experimental.list_physical_devices('GPU'))>1:
    strategy = tf.distribute.MirroredStrategy()
else:
    strategy = tf.distribute.get_strategy()

In [5]:
# create the generators
train_generator, validation_generator = create_generators(args)

num_classes = train_generator.num_classes()
num_anchors = train_generator.num_anchors

# total steps per epoch
if args.steps_per_epoch<1:
    args.steps_per_epoch = len(train_generator)
validation_steps = len(validation_generator)

In [6]:
# if to use tfrecords
if args.use_tfrecords:
    from generators.tfrecords import create_tfrecords, get_loader
    from os.path import exists, join

    if args.dataset_type == 'pascal':
        data_path = args.pascal_path
    elif args.dataset_type == 'coco':
        data_path = args.coco_path
    elif args.dataset_type =='csv':
        data_path = os.path.split(args.base_dir_train)
        data_path = os.path.join(*data_path[:-1])
    else:
        raise Exception('Not implemented yet! Try not using tfrecords option...')
    path_tfrecords = os.path.join(data_path, f'tfrecords_cat_dog_phi{args.phi}')
    os.makedirs(path_tfrecords, exist_ok=True)

    # create tfrecords files
    if not glob(join(path_tfrecords, 'train*.tfrec')):
        print('Creating tfrecords for train data...')
        create_tfrecords(path_tfrecords, 'train', train_generator, repetitions=1)

    if not glob(join(path_tfrecords, 'val*.tfrec')):
        print('Creating tfrecords for validation data...')
        create_tfrecords(path_tfrecords, 'val', validation_generator, repetitions=1)

    # get tfrecords loaders
    train_generator = get_loader(path_tfrecords, 'train', args.batch_size)
    validation_generator = get_loader(path_tfrecords, 'val', args.batch_size)

if not args.compute_val_loss:
    validation_generator = None
elif args.compute_val_loss and validation_generator is None:
    raise ValueError('When you have no validation data, you should not specify --compute-val-loss.')

In [7]:
with strategy.scope():
    model, prediction_model = efficientdet(args.phi,
                                           num_classes=num_classes,
                                           num_anchors=num_anchors,
                                           weighted_bifpn=args.weighted_bifpn,
                                           freeze_bn=args.freeze_bn,
                                           detect_quadrangle=args.detect_quadrangle
                                           )

# load pretrained weights
if args.snapshot:
    if args.snapshot == 'imagenet':
        model_name = 'efficientnet-b{}'.format(args.phi)
        file_name = '{}_weights_tf_dim_ordering_tf_kernels_autoaugment_notop.h5'.format(model_name)
        file_hash = WEIGHTS_HASHES[model_name][1]
        weights_path = keras.utils.get_file(file_name,
                                            BASE_WEIGHTS_PATH + file_name,
                                            cache_subdir='models',
                                            file_hash=file_hash)
        model.load_weights(weights_path, by_name=True)
    else:
        print('Loading model, this may take a second...')
        model.load_weights(args.snapshot)

# freeze backbone layers
if args.freeze_backbone:
    # 227, 329, 329, 374, 464, 566, 656
    for i in range(1, [227, 329, 329, 374, 464, 566, 656][args.phi]):
        model.layers[i].trainable = False

In [8]:
# compile model
if args.loss=='l1':
    regression_loss = smooth_l1_quad() if args.detect_quadrangle else smooth_l1()
else:
    regression_loss = iou_loss(mode=args.loss, phi=args.phi,\
                               weight=args.regression_weight,\
                               freeze_iterations=args.freeze_iterations)

with strategy.scope():
    if 'piou' in args.loss:
        optimizer = Adam(lr=args.lr, epsilon=1e-3, decay=1e-4) #, clipvalue=10.
    else:
        optimizer = Adam(lr=args.lr)
    model.compile(optimizer=optimizer, loss={
        'regression': regression_loss,
        'classification': focal()
    }, )

In [ ]:
# start training
model.fit(
    train_generator,
    steps_per_epoch=args.steps_per_epoch,
    epochs=args.epochs,
    #callbacks=create_callbacks(
    #    model,
    #    prediction_model,
    #    validation_generator,
    #    args,
    #),
    #validation_data=validation_generator,
    #validation_steps=validation_steps
)

Epoch 1/200
515/515 [==============================] - 239s 321ms/step - loss: 2.4901 - classification_loss: 0.8147 - regression_loss: 1.6753
Epoch 2/200
515/515 [==============================] - 177s 343ms/step - loss: 0.9070 - classification_loss: 0.2914 - regression_loss: 0.6156
Epoch 3/200
515/515 [==============================] - 182s 353ms/step - loss: 0.6716 - classification_loss: 0.2149 - regression_loss: 0.4567
Epoch 4/200
515/515 [==============================] - 182s 353ms/step - loss: 0.5281 - classification_loss: 0.1721 - regression_loss: 0.3560
Epoch 5/200
515/515 [==============================] - 183s 355ms/step - loss: 0.4312 - classification_loss: 0.1402 - regression_loss: 0.2911
Epoch 6/200
515/515 [==============================] - 184s 357ms/step - loss: 0.3626 - classification_loss: 0.1152 - regression_loss: 0.2474
Epoch 7/200
515/515 [==============================] - 184s 356ms/step - loss: 0.3138 - classification_loss: 0.0946 - regression_loss: 0.2191
Epoch 

In [ ]:
import matplotlib.pyplot as plt
from utils.anchors import anchors_for_shape
from layers import RegressBoxes

In [ ]:
# BATCH SIZE MUST BE 1
i = np.random.randint(0,len(train_generator))
x,(y_class, y_regress) = train_generator[i]

y_regress.shape

In [ ]:
y_regress[...,:-1].max(), y_regress[...,:-1].min()

In [ ]:
boxes_pred, scores, labels = prediction_model.predict(x)

# select those detections
boxes_pred = boxes_pred[scores>0.5]

boxes_pred.shape, boxes_pred[...,-1]

In [ ]:
# post process image
image = np.copy(np.squeeze(x))
image *= [0.229, 0.224, 0.225]
image += [0.485, 0.456, 0.406]
image = np.uint8(image*255)
image.shape

In [ ]:
# separate target and state
regression_target = y_regress[:, :, :-1]
anchor_state = y_regress[:, :, -1]

# convert to boxes values: xmin, ymin, xmax, ymax, angle
anchors = train_generator.anchors
anchors_input = np.expand_dims(anchors, axis=0)
regression_target = RegressBoxes()([anchors_input, regression_target])

# filter out "ignore" anchors
indices = tf.where(keras.backend.equal(anchor_state, 1))
boxes_true = tf.gather_nd(regression_target, indices)

boxes_true.shape

In [ ]:
plt.figure(figsize=(15,15))
plt.imshow(image)
current_axis = plt.gca()

for b in boxes_true:
    xmin, ymin, xmax, ymax, angle = list(map(int, b))
    cx = (xmax+xmin)/2; cy = (ymax+ymin)/2
    w = (xmax-xmin); h = (ymax-ymin)
    
    xmin = cx - 1 / 2 * (-h * np.sin(angle) + w * np.cos(angle))
    ymin = cy - 1 / 2 * (h * np.cos(angle) + w * np.sin(angle))
    
    current_axis.add_patch(
        plt.Rectangle((xmin, ymin), w, h, angle=angle * 180 / np.pi, color='red', fill=False, linewidth=2)
    )
    
for b in boxes_pred:
    xmin, ymin, xmax, ymax, angle = list(map(int, b))
    cx = (xmax+xmin)/2; cy = (ymax+ymin)/2
    w = (xmax-xmin); h = (ymax-ymin)
    
    xmin = cx - 1 / 2 * (-h * np.sin(angle) + w * np.cos(angle))
    ymin = cy - 1 / 2 * (h * np.cos(angle) + w * np.sin(angle))
    
    current_axis.add_patch(
        plt.Rectangle((xmin, ymin), w, h, angle=angle * 180 / np.pi, color='blue', fill=False, linewidth=2)
    )